# oral_dataloading

> How to load isic datasets for supervised learning, and SSL

Important: Note that have to download the data first from 

 

In [ ]:
#| default_exp oralcancer_dataloading

In [ ]:
#| hide
from nbdev.showdoc import *
from fastcore.test import *

In [ ]:
#| export
import torch
from fastai.vision.all import *
# from self_supervised.augmentations import *
# from self_supervised.layers import *
from base_rbt.utils import *
import re
import os


Unzip example 

In [ ]:
#| hide

# def list_files_in_directory(directory):
#     # This function will list all files in the given directory and return them as a set
#     return set(os.listdir(directory))

# def check_for_overlap(train_dir, test_dir):
#     # List all files in training and testing directories
#     train_files = list_files_in_directory(train_dir)
#     test_files = list_files_in_directory(test_dir)

#     # Check for overlap
#     overlap = train_files & test_files  # This will be the intersection of both sets
#     if overlap:
#         print(f"Overlap detected: {overlap}")
#     else:
#         print("No overlap between training and testing datasets.")

# # Define directories with corrected folder names and handling spaces
# base_dir = '/content/Oral_Cancer_Data'
# train_cancer_dir = os.path.join(base_dir, 'train', 'train', 'CANCER')
# train_non_cancer_dir = os.path.join(base_dir, 'train', 'train', 'NON CANCER')  # Handling space in directory name
# test_cancer_dir = os.path.join(base_dir, 'test', 'test', 'CANCER')
# test_non_cancer_dir = os.path.join(base_dir, 'test', 'test', 'NON CANCER')  # Handling space in directory name

# # Check for overlap in CANCER category
# check_for_overlap(train_cancer_dir, test_cancer_dir)

# # Check for overlap in NON CANCER category
# check_for_overlap(train_non_cancer_dir, test_non_cancer_dir)


Main dataloader functions:

In [ ]:
#| export

def label_func(x):
    # Function to extract the label from the parent directory name
    return x.parent.name

def get_supervised_oralcancer_train_dls(bs, 
                                        dataset_dir, 
                                        size=256, 
                                        device='cpu', 
                                        pct_dataset=1.0, 
                                        num_workers=12):
    train_dir = os.path.join(dataset_dir, "train", "train")  # Adjust for the additional directory layer
    
    # Get image files from the training directory
    fnames = get_image_files(train_dir)

    # Apply subset size
    n = int(len(fnames) * pct_dataset)
    fnames = fnames[:n]

    # Data transformations
    item_tfms = [Resize(size)]

    # Create the DataLoader
    dls = ImageDataLoaders.from_path_func(
        path=train_dir,
        fnames=fnames,
        label_func=label_func,
        bs=bs,
        item_tfms=item_tfms,
        valid_pct=0.0,  # No validation split, using all for training
        device=device,
        num_workers=num_workers * (device == 'cuda')
    )

    return dls

def get_supervised_oralcancer_test_dls(bs, 
                                       dataset_dir, 
                                       size=256, 
                                       device='cpu', 
                                       pct_dataset=1.0, 
                                       num_workers=12):
    test_dir = os.path.join(dataset_dir, "test", "test")  # Adjust for the additional directory layer
    
    # Get image files from the testing directory
    fnames = get_image_files(test_dir)

    # Apply subset size
    n = int(len(fnames) * pct_dataset)
    fnames = fnames[:n]

    # Data transformations
    item_tfms = [Resize(size)]

    # Create the DataLoader
    dls = ImageDataLoaders.from_path_func(
        path=test_dir,
        fnames=fnames,
        label_func=label_func,
        bs=bs,
        item_tfms=item_tfms,
        valid_pct=0,  # No validation split for the test set
        device=device,
        drop_last=False,
        num_workers=num_workers * (device == 'cuda')
    )

    return dls


def get_bt_oralcancer_train_dls(bs,size,device,pct_dataset=1.0,num_workers=12):
    #NOTE: assume unzip like: !unzip -q -o "/content/drive/My Drive/dermnet.zip" -d "/content/drive/My Drive/DermNetDataset"

    dataset_dir = "/content/Oral_Cancer_Data"  #hardcode for SSL.

    return get_supervised_oralcancer_train_dls(bs, 
                                        dataset_dir, 
                                        size=size, 
                                        device=device, 
                                        pct_dataset=pct_dataset, 
                                        num_workers=num_workers)

    

    return dls

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()